# Uchazeč: data o uchazečích o studium na pedagogických fakultách

Od MŠMT máme dva soubory .dbf. Je trochu pain s nimi pracovat.
- Stata mi to nenačte - nejspíš je to špatná verze
- Excel také ne - akorát předstane reagovat
- Python: pomocí balíku dbf lze data konvertovat do csv, potom lze načíst jako dataframe (jen je potřeba specifikovat kódování)

Data jsou popsaná na https://dsia.msmt.cz/uch2022b.html - většina proměnných má vlastní číselníky, je potřeba to poprojovat.
- Dokážu načítat číselníky přímo z webu? Na to byl nějaký jednoduchý mechanismus, ne?

Co chci zkusit?
- Převeď soubory do .csv
- Stáhni .dbf číselníky
- Načti číselníky z webových tabulek (nebo xml?)
- Zmerguj všechno dohromady a vytvoř překonvertovaná data

## Importy

In [1]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import stata_setup
stata_setup.config('/usr/local/stata17', 'be')
from pystata import stata
import dbf


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       BE—Basic Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user , expiring  8 Jul 2023
Serial number: 501709392641
  Licensed to: Tomas Protivinsky
               CERGE-EI

Notes:
      1. Unicode is supported; see help unicode_advice.


## Načtení dat

In [2]:
data_root = '/mnt/d/projects/idea/data'
path17 = 'uchazec/0022MUCH17P'
path21 = 'uchazec/0022MUCH21P'

In [ ]:
# konverze -- nějakou chvilu trvá

# with dbf.Table(f'{data_root}/{path17}.dbf') as table:
#     dbf.export(table, f'{data_root}/{path17}.csv')

# with dbf.Table(f'{data_root}/{path21}.dbf') as table:
#     dbf.export(table, f'{data_root}/{path21}.csv')

In [3]:
# načti data z roku 2017
df = pd.read_csv(f'{data_root}/{path17}.csv', encoding='cp1250')

/tmp/ipykernel_356/2773192003.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path17}.csv', encoding='cp1250')


In [6]:
df.head(100).to_csv('temp/uch17.csv')

## Číselníky

In [8]:
df.columns

Index(['RDAT', 'RID', 'ROD_KOD', 'STATB', 'OBECB', 'PSCB', 'STAT', 'ODHL',
       'IZOS', 'OBSS', 'RMAT', 'TYP_ST', 'FORMA_ST', 'PROGRAM', 'OBOR1',
       'OBOR2', 'OBOR3', 'OBOR4', 'OBOR5', 'APRO1', 'APRO2', 'APRO3',
       'DAT_REG', 'VYPR', 'DAT_VYPR', 'ZAPS', 'DAT_ZAPS', 'CHYV'],
      dtype='object')

In [10]:
foo = pd.read_html('https://dsia.msmt.cz/uch2022b.html')
popis = foo[1].iloc[2:]
popis.columns = foo[1].iloc[1]
popis.to_csv(f'{data_root}/uchazec/popis.csv')
popis.to_csv(f'temp/uchazec/popis.csv')

In [16]:
popis

1,pořadové číslo,Název položky,typ,délka,Popis položky,kontrola
2,1.,RDAT,C,8,rozhodné datum sběru,NaN
3,2.,RID,C,5,identifikátor fakulty,registrace
4,3.,RODC,C,10,rodné číslo,"modulo, tvar,"
5,4.,STATB,C,3,stát trvalého pobytu,číselníky
6,5.,OBECB,C,6,obec trvalého pobytu,číselníky
7,6.,PSCB,C,5,PSČ trvalého pobytu,číselník
8,7.,STAT,C,3,státní příslušnost uchazeče,číselník
9,8.,ODHL,C,1,odkud se uchazeč hlásí,"číselník, RMAT"
10,9.,IZOS,C,9,identifikátor střední školy,"registrace, RMAT"
11,10.,OBSS,C,8,obor studia na střední škole,"registrace, IZOS, RMAT"


In [12]:
# státy - číselník AAST - sloupce STATB, STAT
aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
aast.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None


In [13]:
# pscobec - sloupce OBECB, PSCB
# reálně je nejspíš nepotřebujeme

# with dbf.Table(f'{data_root}/uchazec/pscobec.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/pscobec.csv')
    
pscobec = pd.read_csv(f'{data_root}/uchazec/pscobec.csv', encoding='cp1250')
pscobec.head()

,KOD,NAZEV,OKRES,NAZOKRES,NAZKRAJ,PSC,POSTA,LAU1
0,500011,Želechovice nad Dřevnicí,3705,Zlín,Zlínský kraj,76311,Želechovice nad Dřevnicí,CZ0724
1,500020,Petrov nad Desnou,3809,Šumperk,Olomoucký kraj,78814,Rapotín,CZ0715
2,500020,Petrov nad Desnou,3809,Šumperk,Olomoucký kraj,78816,Petrov nad Desnou,CZ0715
3,500046,Libhošť,3804,Nový Jičín,Moravskoslezský kraj,74257,Libhošť,CZ0804
4,500062,Krhová,3810,Vsetín,Zlínský kraj,75663,Krhová,CZ0723


In [15]:
# odkud se uchazeč hlásí - číselník MCPP - sloupec ODHL
mcpp_xml = 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml'
mcpp = pd.read_xml(mcpp_xml, encoding='cp1250', xpath='./veta')
mcpp

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,MCPP,1,Základní škola,Základní škola,15.4.1993,31.12.2999,NaN,NaN
1,MCPP,2,Střední škola,Střední škola,15.4.1993,31.12.2999,NaN,NaN
2,MCPP,3,Zaměstnání,Zaměstnání,15.4.1993,31.12.2999,NaN,NaN
3,MCPP,4,Domácnost,Domácnost,15.4.1993,31.12.2999,NaN,NaN
4,MCPP,6,Vyšší odborná škola,Vyšší odborná škola,30.6.1997,31.12.2999,NaN,NaN
5,MCPP,7,Vysoká škola,Vysoká škola,1.7.1997,31.12.2999,NaN,NaN
6,MCPP,8,Přichází přes DZS,Přichází přes Dům zahraničních služeb,21.1.1994,31.12.2999,NaN,NaN
7,MCPP,9,Jiné,Jiné,15.4.1993,31.12.2999,NaN,NaN
8,MCPP,C,Celoživ.vzdělávání VŠ,Program celoživotního vzdělávání na VŠ,1.1.2013,31.12.2999,NaN,NaN


In [17]:
# typ studia - číselník PASP - sloupec TYP_ST
pasp_xml = 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml'
pasp = pd.read_xml(pasp_xml, encoding='cp1250', xpath='./veta')
pasp

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,PASP,B,Bakalářský,Bakalářský,1.9.1999,31.12.2999,NaN,NaN
1,PASP,D,Doktorský,Doktorský,1.9.1999,31.12.2999,NaN,NaN
2,PASP,M,Magisterský,Magisterský,1.9.1999,31.12.2999,NaN,NaN
3,PASP,N,Magist.navaz.na bakalář.,Magisterský navazující na bakalářský,1.9.1999,31.12.2999,NaN,NaN


In [19]:
# forma studia - číselník PAFS - sloupec FORMA_ST
pafs_xml = 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml'
pafs = pd.read_xml(pafs_xml, encoding='cp1250', xpath='./veta')
pafs

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,PAFS,D,Distanční,Distanční,1.9.1999,31.12.2999,NaN,NaN
1,PAFS,K,Kombinované,Kombinované,1.9.1999,31.12.2999,NaN,NaN
2,PAFS,P,Prezenční,Prezenční,1.9.1999,31.12.2999,NaN,NaN


In [21]:
# obor - číselník AKVO - sloupce OBOR[1-5]
akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
akvo.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AKVO,1101R006,Deskriptivní geometrie,Deskriptivní geometrie,1.1.2003,31.12.2999,None,None
1,AKVO,1101R008,Diskrétní matematika,Diskrétní matematika,1.1.2003,31.12.2999,None,None
2,AKVO,1101R014,Matematická analýza,Matematická analýza,1.1.2003,31.12.2999,11047,None
3,AKVO,1101R016,Matematika,Matematika,1.1.2003,31.12.2999,11617,None
4,AKVO,1101R018,"Matem.,aplikace(analýza)",Matematika a její aplikace (zaměření matematic...,1.1.2003,31.12.2999,None,None


In [105]:
akvo.shape

(6218, 8)

In [23]:
# výsledek přijímacího řízení - číselník MCPR - sloupec VYPR
mcpr_xml = 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml'
mcpr = pd.read_xml(mcpr_xml, encoding='cp1250', xpath='./veta')
mcpr

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,MCPR,10,Přijat na zákl.přij.zk.,Přijat na základě přijímací zkoušky,1.1.2001,31.12.2999,NaN,NaN
1,MCPR,11,Přijat bez přij.zkoušky,Přijat bez přijímací zkoušky,1.1.2001,31.12.2999,NaN,NaN
2,MCPR,12,"Přijat po odvol.,dodateč.","Přijat dodatečně (po odvolání, po přezkumném ř...",1.1.2001,31.12.2999,NaN,NaN
3,MCPR,14,Přijat přestupem,Přijat z předchozího vysokoškolského studia (p...,1.11.2019,31.12.2999,NaN,NaN
4,MCPR,15,Přijat z CŽV,Přijat z kurzů celoživotního vzdělávání,1.11.2019,31.12.2999,NaN,NaN
5,MCPR,20,Nepřijat-neprosp.přij.zk.,Nepřijat pro neprospěch u přijímací zkoušky,1.1.2001,31.12.2999,NaN,NaN
6,MCPR,22,Nepřijat-nedost.kapac.ob.,Nepřijat pro nedostačující kapacitu oboru/stud...,1.1.2001,31.12.2999,NaN,NaN
7,MCPR,23,Nepřijat-nedost.vzdělání,Nepřijat pro nedoložení požadovaného předchozí...,1.11.2019,31.12.2999,NaN,NaN
8,MCPR,24,Nepřijat-další podmínky,Nepřijat pro nesplnění dalších podmínek přijetí,1.11.2019,31.12.2999,NaN,NaN
9,MCPR,30,"Př.řiz.zrušeno,nedost.uch",Přijímací řízení zrušeno pro nedostatek uchazečů,5.11.2009,31.12.2999,NaN,NaN


In [24]:
# výsledek zápisu - číselník MCZR - sloupec ZAPS
mczr_xml = 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml'
mczr = pd.read_xml(mczr_xml, encoding='cp1250', xpath='./veta')
mczr

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,MCZR,0,Uchazeč nebyl přijat,Uchazeč nebyl přijat,15.4.1993,31.12.2999,NaN,NaN
1,MCZR,1,Uchazeč se zapsal,Uchazeč se zapsal,15.4.1993,31.12.2999,NaN,NaN
2,MCZR,2,Uchazeč se nezapsal,Uchazeč se nezapsal,15.4.1993,31.12.2999,NaN,NaN


In [25]:
# aprobace - číselník AAAP - sloupce APRO[1-3]
aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
aaap.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAAP,1,Předměty pro 1.stupeň ZŠ,Předměty a metodiky předmětů pro 1. stupeň zák...,15.4.1993,31.12.2999,1ST,NaN
1,AAAP,2,Český jazyk,Český jazyk,15.4.1993,31.12.2999,CJ,NaN
2,AAAP,3,Slovenský jazyk,Slovenský jazyk,15.4.1993,31.12.2999,SJ,NaN
3,AAAP,4,Maďarský jazyk,Maďarský jazyk,15.4.1993,31.12.2999,MJ,NaN
4,AAAP,5,Polský jazyk,Polský jazyk,15.4.1993,31.12.2999,PJ,NaN


In [26]:
# ok, toto vypadá dobře
df['APRO1'].unique()

array(['   ', '07 ', '23 ', '32 ', '02 ', '45 ', '80 ', '09 ', '37 ',
       '33 ', '30 ', '26 ', '08 ', '29 ', '50 ', '06 ', '12 ', '22 ',
       '16 ', '21 ', '62 ', '05 ', '35 ', '15 ', '11 '], dtype=object)

## Číselníky z programů...

Lze stáhnout kontrolní program z https://dsia.msmt.cz/program.html - ten obsahuje složku "cisel", v ní jsou ty chybějící .dbf číselníky...

In [27]:
# fakulty - číselník regpro.dbf - sloupec RID

# with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv')
    
regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
regpro.head()

,RID,ZAR_PLN,VS_PLN,ZAR_NAZ,PSC,MISTO,ULICE,CP,SUZEMI,UZEMI,VS_ZKR,DAT_VZN,DAT_ZAH,DAT_AKT,DAT_KON,ID_FNAM,ID_SUCH
0,12520,Fakulta rybářství a ochrany vod ...,Jihočeská univerzita v Českých Budějovicích ...,Fakulta rybářství a ochrany vod,389 25,Vodňany,Zátiší 728/II,,3307,CZ0316,JU v Českých Budějovicích,2009-09-01,2009-09-01,None,None,BP0,1252
1,19510,Fakulta veřejných politik v Opavě ...,Slezská univerzita v Opavě ...,Fakulta veřejných politik v Opavě,746 01,Opava,Olbrichova 625/25,,3806,CZ0815,SU v Opavě,2008-10-01,2008-10-01,2010-06-21,None,IC0,1951
2,41320,Fakulta lesnická a dřevařská ...,Česká zemědělská univerzita v Praze ...,Fakulta lesnická a dřevařská,165 21,Praha 6 - Suchdol,Kamýcká 1176,,3106,CZ0116,ČZU v Praze,2007-07-01,None,2009-04-21,None,RF0,4132
3,15120,Fakulta zdravotnických věd ...,Univerzita Palackého v Olomouci ...,Fakulta zdravotnických věd,771 26,Olomouc 1,tř. Svobody 8,,3805,CZ0712,UP v Olomouci,None,None,None,None,ER0,1512
4,23310,Fakulta zdravotnických studií ...,Západočeská univerzita v Plzni ...,Fakulta zdravotnických studií,306 14,Plzeň,Univerzitní 8,,3405,CZ0323,ZČU v Plzni,2008-07-08,2008-07-08,None,None,LY0,2331


In [28]:
regpro.shape

(210, 17)

In [29]:
# studijní program - číselník program.dbf - sloupec PROGRAM
# toto je taky trochu mess, ale jde to spojit

# with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/program.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/program.csv')
    
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.head()

,KOD,NAZEV,RID
0,B0111A190001,Pedagogika ...,14210
1,B0111A190002,Sociální pedagogika a volný čas ...,14410
2,B0111A190003,Andragogika ...,15210
3,B0111A190004,Andragogika a personální řízení ...,11210
4,B0111A190005,Sociální pedagogika a poradenství ...,14210


In [147]:
program.shape

(2144, 3)

In [30]:
program['KOD'].str[:5]

0       B0111
1       B0111
2       B0111
3       B0111
4       B0111
        ...  
2139    B0923
2140    B0715
2141    B0715
2142    B0716
2143    B0114
Name: KOD, Length: 2144, dtype: object

In [151]:
df['PROGRAM'].drop_duplicates().isin(program['KOD'].str[:5])

0         True
1         True
2         True
3         True
19        True
          ... 
60817     True
66911     True
111461    True
177473    True
202108    True
Name: PROGRAM, Length: 223, dtype: bool

In [158]:
df['PROGRAM'].drop_duplicates().isin(program['KOD'].str.replace(' ', '')).sum()

223

In [154]:
df['PROGRAM'].drop_duplicates()

0         B6708
1         B6107
2         M5103
3         M5111
19        B2612
          ...  
60817     B3973
66911     B2831
111461    M2301
177473    B1513
202108    B3964
Name: PROGRAM, Length: 223, dtype: object

In [153]:
program.to_csv('temp/program.csv')

In [159]:
# střední školy - číselník izobrmat.dbf - sloupce IZOS, OBSS

with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.dbf') as table:
    dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.csv')
    
izobrmat = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.csv', encoding='cp1250')
izobrmat.head()

,IZO,OBOR,RMAT
0,10189,2343L506,1999
1,10189,2343L506,2000
2,10189,2343L506,2002
3,10189,2343L506,2003
4,10189,2345L001,2001


In [160]:
izobrmat.shape

(117926, 3)

In [165]:
# střední školy - číselník typss.dbf - k čemu to je?

# with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/typss.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/typss.csv')
    
typss = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/typss.csv', encoding='cp1250')
typss

,IZO,ZARIZENI,MISTO,ULICE,CP,PSC,TYP,ZRIZ,AKT
0,333333333,ISŠ,,,,,311,,3
1,444444444,SOU,,,,,311,,3
2,555555555,gymnázia,,,,,321,,3
3,666666666,SOŠ,,,,,331,,3
4,777777777,školy ministerstva obrany,,,,,,,3
5,888888888,školy ministerstva vnitra,,,,,,,3
6,999999999,zahraniční školy,,,,,,,3


In [164]:
typss.shape

(7, 9)

## Střední školy -- mess...

In [33]:
# střední školy -- to je mess

# for y in range(1999, 2023):
#     with dbf.Table(f'{data_root}/uchazec/stredni-skoly/m{y}.dbf') as table:
#         dbf.export(table, f'{data_root}/uchazec/stredni-skoly/m{y}.csv')

In [34]:
foo = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m2017.csv', encoding='cp1250')

In [35]:
foo.head()

,RED_IZO,IZO,ZAR_PLN,ZAR_ZKR,MISTO,ULICE,CP,PSC,VUSC,OBOR,OBOR_NAZ,PLN_NAZ
0,600006654,13757,"Střední škola mediální grafiky a tisku, s.r.o....","SŠ mediální grafiky a tisku, s.r.o.",Praha 9,Beranových 695,,19900,CZ0109,3452L01,Tiskař na polygr.strojích,Tiskař na polygrafických strojích ...
1,600006654,13757,"Střední škola mediální grafiky a tisku, s.r.o....","SŠ mediální grafiky a tisku, s.r.o.",Praha 9,Beranových 695,,19900,CZ0109,3453L01,Reprod.grafik pro média,Reprodukční grafik pro média ...
2,600014614,53155,"Střední škola gastronomie, hotelnictví a lesni...",SŠ gastronomie hotelnictví a lesnictví,Bzenec,náměstí Svobody 318,,69681,CZ0645,1601M01,Ekologie a životní prost.,Ekologie a životní prostředí ...
3,600014614,53155,"Střední škola gastronomie, hotelnictví a lesni...",SŠ gastronomie hotelnictví a lesnictví,Bzenec,náměstí Svobody 318,,69681,CZ0645,2942M01,Analýza potravin,Analýza potravin ...
4,600014614,53155,"Střední škola gastronomie, hotelnictví a lesni...",SŠ gastronomie hotelnictví a lesnictví,Bzenec,náměstí Svobody 318,,69681,CZ0645,4145L51,Mechanizace zemědělství,Mechanizace zemědělství a lesního hospodářství...


In [36]:
foo.shape

(3211, 12)

In [ ]:
# mám zde všechny školy z dat?

In [135]:
df.shape

(253584, 28)

In [142]:
# kolik máme cizinců?
(pd.to_numeric(df['STATB'], errors='coerce') != 203).sum()

36323

In [143]:
df['IZOS'].str.contains('999999').sum()

38223

In [39]:
df['IZOS'].unique()

array(['999999840', '999999368', '999999112', ..., '999999136',
       '999999408', '999999162'], dtype=object)

In [130]:
cz_izos = pd.to_numeric(df['IZOS'][~df['IZOS'].str.contains('999999')].drop_duplicates().sort_values()[1:]).reset_index(drop=True)

In [131]:
cz_izos

0               0
1             203
2           10189
3           13757
4           43001
          ...    
2588       848956
2589        87700
2590        87904
2591    888888888
2592     99999398
Name: IZOS, Length: 2593, dtype: int64

In [125]:
df['IZOS'].drop_duplicates().str.contains('999999').sum()

183

In [117]:
izos = pd.to_numeric(df['IZOS'].drop_duplicates().sort_values().iloc[1:])

In [119]:
izos.shape

(2776,)

In [123]:
izos = izos[izos < 999999000]

In [124]:
izos

7516             0
967            203
36956        10189
940          13757
4076         43001
            ...   
29208     99999931
115365    99999940
198279    99999950
35761     99999951
53897     99999970
Name: IZOS, Length: 2598, dtype: int64

In [41]:
foo['RED_IZO'].sort_values()

2015    600001431
2896    600001873
706     600004520
705     600004520
704     600004520
          ...    
536     691007551
1222    691009619
1221    691009619
1220    691009619
1219    691009619
Name: RED_IZO, Length: 3211, dtype: int64

In [42]:
foo['IZO'].sort_values()

0           13757
1           13757
2           53155
3           53155
4           53155
          ...    
3088    200000071
2910    250000016
2911    250000016
2909    250000016
3210    250000059
Name: IZO, Length: 3211, dtype: int64

In [47]:
pd.Series(df['IZOS'].unique()).isin(foo['RED_IZO']).sum()

0

In [65]:
num_izos = pd.to_numeric(df['IZOS'], errors='coerce')

In [68]:
num_izos.isin(foo['RED_IZO'])

0         False
1         False
2         False
3         False
4         False
          ...  
253579    False
253580    False
253581    False
253582    False
253583    False
Name: IZOS, Length: 253584, dtype: bool

In [67]:
num_izos.isin(foo['RED_IZO']).sum()

225

In [70]:
df['OBSS'].unique()

array(['        ', '7941K801', '1601M002', ..., '  6324 M', '53-01-6 ',
       '646103  '], dtype=object)

In [78]:
df['OBSS'].unique().shape

(1398,)

In [75]:
np.sort(foo['OBOR'].unique())

array(['1601M01 ', '1601N01 ', '1602M01 ', '1820M01 ', '2142M01 ',
       '2143L01 ', '2143M01 ', '2144L01 ', '2341M01 ', '2341N02 ',
       '2341N03 ', '2341N06 ', '2341N07 ', '2341N08 ', '2343L51 ',
       '2344L01 ', '2345L01 ', '2345L02 ', '2345M01 ', '2345N01 ',
       '2345N02 ', '2345N03 ', '2362L01 ', '2369L01 ', '2641L01 ',
       '2641L51 ', '2641L52 ', '2641M01 ', '2641N01 ', '2641N02 ',
       '2641N03 ', '2641N06 ', '2645M01 ', '2647N02 ', '2647N03 ',
       '2647N04 ', '2647N06 ', '2647N07 ', '2647N09 ', '2647N13 ',
       '2647N14 ', '2647N15 ', '2647N16 ', '2647N17 ', '2647N18 ',
       '2647N19 ', '2832N01 ', '2842L01 ', '2844M01 ', '2846M01 ',
       '2941L51 ', '2941M01 ', '2941N01 ', '2941N03 ', '2942M001',
       '2942M01 ', '3141M004', '3141M01 ', '3143L01 ', '3143M01 ',
       '3241M01 ', '3331N01 ', '3341L01 ', '3342L51 ', '3342M01 ',
       '3441M01 ', '3442M01 ', '3452L01 ', '3453L01 ', '3456L01 ',
       '3457L01 ', '3641N03 ', '3641N04 ', '3641N05 ', '3641N0

In [77]:
pd.Series(df['OBSS'].unique()).isin(foo['OBOR']).sum()

293

In [81]:
pd.Series(df['OBSS'].unique()).str.replace(' ', '').values

array(['', '7941K801', '1601M002', ..., '6324M', '53-01-6', '646103'],
      dtype=object)

In [84]:
pd.Series(df['OBSS'].unique()).str.replace(' ', '').unique().shape

(1389,)

In [82]:
pd.Series(df['OBSS'].unique()).str.replace(' ', '').isin(foo['OBOR']).sum()

16

In [85]:
pd.Series(df['OBSS'].unique()).str.replace(' ', '').isin(foo['OBOR'].str.replace(' ', '')).sum()

297

Ok, toto je mess -> většina škol v číselníku není (všechny začínající 999999...), ani většina oborů tam není. Chci zkusit mergovat všechny roky dohromady?

In [86]:
li = []

for y in range(1999, 2023):
    pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
    li.append(pdf)

In [91]:
foo = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

In [92]:
foo.shape

(34994, 13)

In [132]:
foo['IZO']

0            10189
1            10189
2            13757
3            13757
4            53155
           ...    
34989    181078988
34990    181088231
34991    181096790
34992    181113601
34993    181120585
Name: IZO, Length: 34994, dtype: int64

In [100]:
pd.to_numeric(df['IZOS'][:5]).isin(foo['IZO'])

0    False
1    False
2    False
3    False
4    False
Name: IZOS, dtype: bool

In [133]:
cz_izos.isin(foo['IZO'])

0       False
1       False
2        True
3        True
4       False
        ...  
2588     True
2589    False
2590     True
2591    False
2592    False
Name: IZOS, Length: 2593, dtype: bool

In [134]:
cz_izos.isin(foo['IZO']).sum()

1919

Ok, toto je nakonec docela slušné -> jsem schopen identifikovat 1919 z 2593 SŠ, které nejsou zahraniční (těch je asi 200)

In [162]:
cz_izos.isin(izobrmat['IZO']).sum()

2083

A s jejich číselníkem je to ještě o kousek lepší - 2083 z 2593 SŠ.

## Misc

In [ ]:
pd.read_html('https://dsia.msmt.cz/uch2022b.html')

In [ ]:
!pip install lxml

In [ ]:
!pip install simpledbf

In [ ]:
import pandas as pd
from simpledbf import Dbf5

In [ ]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import stata_setup
stata_setup.config('/usr/local/stata17', 'be')
from pystata import stata

In [ ]:
root = '/mnt/d/projects/idea/data'

In [ ]:
dbf_file = '/mnt/d/projects/idea/data/uchazec/0022MUCH21P.dbf'

In [ ]:
dbf = Dbf5('/mnt/d/projects/idea/data/uchazec/0022MUCH21P.dbf')

In [ ]:
dbf

In [ ]:
dbf.numrec

In [ ]:
dbf.fields

In [ ]:
df = dbf.to_dataframe()

In [ ]:
df

In [ ]:
!ls /mnt/d/projects/idea/data/uchazec/

In [ ]:
!pip install dbf

In [ ]:
import dbf

In [ ]:
foo = dbf.Table(dbf_file)

In [ ]:
foo

In [ ]:
with dbf.Table(dbf_file) as table:
    dbf.export(table, '/mnt/d/projects/idea/data/uchazec/0022MUCH21P.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/mnt/d/projects/idea/data/uchazec/0022MUCH17P.csv', encoding='cp1250')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df[:1000].to_csv('temp/uchazec.csv')

In [ ]:
foo

In [ ]:
df,